**Download the dataset archive of dishes With sideangles and extract it**

# Dataset Download & Prepare files
```

```

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

zip_path = '/content/drive/MyDrive/side_angles.zip'
extract_path = '/content/dataset'

if not os.path.exists(extract_path):
    os.makedirs(extract_path)

os.system(f'unzip -q "{zip_path}" -d "{extract_path}"')

0

In [3]:
!cp -r '/content/drive/MyDrive/nutrition5k/metadata' '/content/dataset/metadata'
!cp -r '/content/drive/MyDrive/nutrition5k/dish_ids' '/content/dataset/dish_ids'

**Model**

In [2]:
import torch
from torch import nn
from torch.optim import Adam
from torchvision.transforms import transforms
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from PIL import Image
import torchvision.models as models
from sklearn.preprocessing import MultiLabelBinarizer
import torch.nn.functional as F
from datetime import datetime
import timm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

# Dataset parsing

In [3]:
metadata_cafe1 = './dataset/metadata/dish_metadata_cafe1.csv'
# metadata_cafe1 = '../../nutrition5k_dataset/metadata/dish_metadata_cafe1.csv'

In [4]:
def parse_file(data):
    data = data.split('\n')
    total = 0
    dishes = []
    df_data = []
    print("lines length: ", len(data))
    for line in data:
        line = line.strip()
        if line == '':
            continue
        line = line.split(',')

        num_ingredients = (len(line) - 6) // 7

        new_dish = {
            'dish_id': line[0],
            'total_calories': float(line[1]),
            'total_mass': float(line[2]),
            'total_fat': float(line[3]),
            'total_carbs': float(line[4]),
            'total_protein': float(line[5]),
        }
        dishes.append(line[0])

        total = total + 1
        for i in range(num_ingredients):
            ingredients = line[6+i*7:6+(i+1)*7]
            ingredient = {
                'ingredient_id': ingredients[0],
                'ingredient_name': ingredients[1],
                'ingredient_mass': float(ingredients[2]),
                'ingredient_calories': float(ingredients[3]),
                'ingredient_fat': float(ingredients[4]),
                'ingredient_carbs': float(ingredients[5]),
                'ingredient_protein': float(ingredients[6])
            }
            df_data.append({**new_dish, **ingredient})
    print("total dishes: ", total)
    return df_data, dishes


def read_and_parse_file(file_path):
    with open(file_path, 'r') as file:
        return parse_file(file.read())


df_data, dishes = read_and_parse_file(metadata_cafe1)
dishes_df = pd.DataFrame(dishes, columns=['dish_id'])

dataset = pd.DataFrame(df_data)

print("total shape", dataset.shape)
print("unique ingredient_ids", dataset['ingredient_id'].unique().shape)

print("unique dishes ids based only on ids from dataset: ",
      dishes_df['dish_id'].unique().shape)
print("unique dishes ids based on all combination ingredient - dish ",
      dataset['dish_id'].unique().shape)

lines length:  4769
total dishes:  4768
total shape (27225, 13)
unique ingredient_ids (211,)
unique dishes ids based only on ids from dataset:  (4768,)
unique dishes ids based on all combination ingredient - dish  (4768,)


In [5]:
def get_top_n_ingredients_by_mass(dataset, N=75):
    skip = ['olive oil', 'salt', 'pepper', 'vinegar', 'coffee',
            'plate only', 'vegetable oil', 'deprecated', 'sugar']

    dataset['ingredient_name'] = dataset['ingredient_name'].str.lower()
    filtered_dataset = dataset[~dataset['ingredient_name'].isin(skip)]

    ingredient_total_mass = filtered_dataset.groupby(
        'ingredient_name')['ingredient_mass'].sum().sort_values(ascending=False)

    top_N_ingredients = ingredient_total_mass.head(N).index.tolist()

    final_dataset = filtered_dataset[filtered_dataset['ingredient_name'].isin(
        top_N_ingredients)]

    return final_dataset


topIngredients = get_top_n_ingredients_by_mass(dataset)

topIngredients['dish_id']

,dish_id
2,dish_1561662216
4,dish_1561662216
5,dish_1561662216
7,dish_1561662216
8,dish_1561662216
...,...
27211,dish_1562691737
27215,dish_1558458496
27216,dish_1568664931
27218,dish_1568664931


In [6]:
top5ingredientsByMass = topIngredients

In [7]:
def create_transforms(input_size=240):

    from torchvision import transforms

    train_transform = transforms.Compose([
        transforms.Resize(input_size + 20),
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

    val_transform = transforms.Compose([
        transforms.Resize(input_size + 20),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

    return train_transform, val_transform

In [8]:
class DishDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None, mlb=None):
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.transform = transform

        dish_groups = dataframe.groupby('dish_id', sort=False)

        top5_ingredients = dataframe.groupby(
            'dish_id')['ingredient_name'].apply(list)

        if mlb is None:
            self.mlb = MultiLabelBinarizer()
            self.ingredient_labels = self.mlb.fit_transform(top5_ingredients)
        else:
            self.mlb = mlb
            self.ingredient_labels = self.mlb.transform(top5_ingredients)

        nutrition_data = dish_groups[[
            'total_fat', 'total_carbs', 'total_protein', 'total_mass', 'total_calories']].first()

        self.fat_densities = nutrition_data['total_fat'] / \
            nutrition_data['total_mass']
        self.carbs_densities = nutrition_data['total_carbs'] / \
            nutrition_data['total_mass']
        self.protein_densities = nutrition_data['total_protein'] / \
            nutrition_data['total_mass']

        self.total_masses = nutrition_data['total_mass'].values
        self.total_calories = nutrition_data['total_calories'].values

        from sklearn.preprocessing import StandardScaler

        self.fat_scaler = StandardScaler()
        self.fat_densities_normalized = self.fat_scaler.fit_transform(
            self.fat_densities.values.reshape(-1, 1)).flatten()

        self.carbs_scaler = StandardScaler()
        self.carbs_densities_normalized = self.carbs_scaler.fit_transform(
            self.carbs_densities.values.reshape(-1, 1)).flatten()

        self.protein_scaler = StandardScaler()
        self.protein_densities_normalized = self.protein_scaler.fit_transform(
            self.protein_densities.values.reshape(-1, 1)).flatten()

        self.mass_scaler = StandardScaler()
        self.total_masses_normalized = self.mass_scaler.fit_transform(
            self.total_masses.reshape(-1, 1)).flatten()

        self.calorie_scaler = StandardScaler()
        self.total_calories_normalized = self.calorie_scaler.fit_transform(
            self.total_calories.reshape(-1, 1)).flatten()

        self.scalers = {
            'fat_scaler': self.fat_scaler,
            'carbs_scaler': self.carbs_scaler,
            'protein_scaler': self.protein_scaler,
            'mass_scaler': self.mass_scaler,
            'calorie_scaler': self.calorie_scaler
        }

        self.dish_ids = list(nutrition_data.index)
        self.dish_to_idx = {dish_id: idx for idx,
                            dish_id in enumerate(self.dish_ids)}

        self.image_paths = []
        self.label_indices = []

        for dish_id in self.dish_ids:
            dish_path = os.path.join(img_dir, str(dish_id), 'frames')
            if not os.path.exists(dish_path):
                continue

            for img_file in os.listdir(dish_path):
                if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    self.image_paths.append(os.path.join(dish_path, img_file))
                    self.label_indices.append(self.dish_to_idx[dish_id])

        print(f"Dataset created with {len(self.mlb.classes_)} ingredients")
        print(f"Using separate heads for fat, carbs, protein densities, total mass, and total calories")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')

        label_idx = self.label_indices[idx]
        ingredient_label = self.ingredient_labels[label_idx]
        fat_label = self.fat_densities_normalized[label_idx]
        carbs_label = self.carbs_densities_normalized[label_idx]
        protein_label = self.protein_densities_normalized[label_idx]
        mass_label = self.total_masses_normalized[label_idx]
        calorie_label = self.total_calories_normalized[label_idx]

        if self.transform:
            image = self.transform(image)

        return (image,
                torch.tensor(ingredient_label, dtype=torch.float32),
                torch.tensor([fat_label], dtype=torch.float32),
                torch.tensor([carbs_label], dtype=torch.float32),
                torch.tensor([protein_label], dtype=torch.float32),
                torch.tensor([mass_label], dtype=torch.float32),
                torch.tensor([calorie_label], dtype=torch.float32))

    def denormalize_predictions(self, fat_pred, carbs_pred, protein_pred, mass_pred, calorie_pred):
        fat_denorm = self.fat_scaler.inverse_transform(
            fat_pred.cpu().numpy().reshape(-1, 1)).flatten()
        carbs_denorm = self.carbs_scaler.inverse_transform(
            carbs_pred.cpu().numpy().reshape(-1, 1)).flatten()
        protein_denorm = self.protein_scaler.inverse_transform(
            protein_pred.cpu().numpy().reshape(-1, 1)).flatten()
        mass_denorm = self.mass_scaler.inverse_transform(
            mass_pred.cpu().numpy().reshape(-1, 1)).flatten()
        calorie_denorm = self.calorie_scaler.inverse_transform(
            calorie_pred.cpu().numpy().reshape(-1, 1)).flatten()
        return fat_denorm, carbs_denorm, protein_denorm, mass_denorm, calorie_denorm

In [9]:
class IngredientClassifier(nn.Module):
    def __init__(self, num_ingredients, model_variant='efficientnetv2_rw_s', freeze_backbone=True, freeze_stages=None):
        super().__init__()

        self.model_variant = model_variant
        self.freeze_backbone = freeze_backbone
        self.freeze_stages = freeze_stages or []

        efficientnetv2_configs = {
            'efficientnetv2_rw_s': {'features': 1280, 'input_size': 384},
            'efficientnetv2_rw_m': {'features': 1280, 'input_size': 416},
            'efficientnetv2_rw_l': {'features': 1280, 'input_size': 480},
        }

        efficientnet_configs = {
            'efficientnet_b0': {'features': 1280, 'input_size': 224},
            'efficientnet_b1': {'features': 1280, 'input_size': 240},
            'efficientnet_b2': {'features': 1408, 'input_size': 260},
            'efficientnet_b3': {'features': 1536, 'input_size': 300},
            'efficientnet_b4': {'features': 1792, 'input_size': 380},
            'efficientnet_b5': {'features': 2048, 'input_size': 456},
            'efficientnet_b6': {'features': 2304, 'input_size': 528},
            'efficientnet_b7': {'features': 2560, 'input_size': 600}
        }

        all_configs = {**efficientnetv2_configs, **efficientnet_configs}

        if model_variant in all_configs:
            config = all_configs[model_variant]
            self.feature_dim = config['features']
            self.input_size = config['input_size']
        else:
            self.feature_dim = 1280
            self.input_size = 224

        try:
            self.backbone = timm.create_model(
                model_variant,
                pretrained=True,
                num_classes=0,
                global_pool=''
            )
            self.feature_dim = self.backbone.num_features
            print(f"Detected feature dimension: {self.feature_dim}")
        except Exception as e:
            print(f"Failed to load {model_variant}: {e}")
            self.backbone = timm.create_model(
                'efficientnet_b0', pretrained=True, num_classes=0, global_pool='')
            self.feature_dim = 1280
            self.input_size = 224
            self.model_variant = 'efficientnet_b0'

        self._apply_freezing()
        self._build_heads(num_ingredients)
        self._print_model_info(num_ingredients)

    def _apply_freezing(self):
        if self.freeze_backbone:
            for param in self.backbone.parameters():
                param.requires_grad = False

        total_params = sum(p.numel() for p in self.backbone.parameters())
        frozen_params = sum(p.numel()
                            for p in self.backbone.parameters() if not p.requires_grad)
        trainable_params = total_params - frozen_params

        print(f"Backbone parameters: {total_params:,}")
        print(
            f"Frozen parameters: {frozen_params:,} ({frozen_params/total_params*100:.1f}%)")
        print(
            f"Trainable parameters: {trainable_params:,} ({trainable_params/total_params*100:.1f}%)")

    def _build_heads(self, num_ingredients):
        self.shared_features = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(self.feature_dim, 4096),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.BatchNorm1d(4096),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.BatchNorm1d(4096)
        )

        self.ingredient_head = nn.Sequential(
            nn.Linear(4096, 2048),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(2048, num_ingredients)
        )

        self.fat_head = nn.Sequential(
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.BatchNorm1d(4096),
            nn.Linear(4096, 1024),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024, 1),
            nn.ReLU()
        )

        self.carbs_head = nn.Sequential(
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.BatchNorm1d(4096),
            nn.Linear(4096, 1024),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024, 1),
            nn.ReLU()
        )

        self.protein_head = nn.Sequential(
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.BatchNorm1d(4096),
            nn.Linear(4096, 1024),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024, 1),
            nn.ReLU()
        )

        self.mass_head = nn.Sequential(
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.BatchNorm1d(4096),
            nn.Linear(4096, 1024),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024, 1),
            nn.ReLU()
        )

        self.calorie_head = nn.Sequential(
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.BatchNorm1d(4096),
            nn.Linear(4096, 1024),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024, 1),
            nn.ReLU()
        )

    def _print_model_info(self, num_ingredients):
        print(f"Model architecture (adapted from paper's multi-task design):")
        print(f"Backbone: {self.model_variant}")
        print(f"Input size: {self.input_size}x{self.input_size}")
        print(f"Shared features: {self.feature_dim} → 4096 → 4096")
        print(f"Ingredient head: 4096 → 2048 → {num_ingredients}")
        print(f"Fat head: 4096 → 4096 → 1024 → 1 (portion-independent density)")
        print(f"Carbs head: 4096 → 4096 → 1024 → 1 (portion-independent density)")
        print(f"Protein head: 4096 → 4096 → 1024 → 1 (portion-independent density)")
        print(f"Mass head: 4096 → 4096 → 1024 → 1")
        print(f"Calorie head: 4096 → 4096 → 1024 → 1")

    def forward(self, x):
        features = self.backbone(x)
        shared_feat = self.shared_features(features)

        ingredient_logits = self.ingredient_head(shared_feat)
        ingredient_probs = torch.sigmoid(ingredient_logits)

        fat_density = self.fat_head(shared_feat)
        carbs_density = self.carbs_head(shared_feat)
        protein_density = self.protein_head(shared_feat)

        total_mass = self.mass_head(shared_feat)
        total_calories = self.calorie_head(shared_feat)

        return ingredient_probs, fat_density, carbs_density, protein_density, total_mass, total_calories

In [10]:
class MultiHeadLoss(nn.Module):
    def __init__(self, ingredient_weight=1.0, fat_weight=1.0, carbs_weight=1.0,
                 protein_weight=1.0, mass_weight=1.0, calorie_weight=1.0):
        super().__init__()
        self.ingredient_weight = ingredient_weight
        self.fat_weight = fat_weight
        self.carbs_weight = carbs_weight
        self.protein_weight = protein_weight
        self.mass_weight = mass_weight
        self.calorie_weight = calorie_weight

        self.bce_loss = nn.BCELoss()
        self.mae_loss = nn.L1Loss()

    def forward(self, ingredient_pred, fat_pred, carbs_pred, protein_pred, mass_pred, calorie_pred,
                ingredient_target, fat_target, carbs_target, protein_target, mass_target, calorie_target):

        ingredient_loss = self.bce_loss(ingredient_pred, ingredient_target)

        fat_loss = self.mae_loss(fat_pred, fat_target)
        carbs_loss = self.mae_loss(carbs_pred, carbs_target)
        protein_loss = self.mae_loss(protein_pred, protein_target)

        mass_loss = self.mae_loss(mass_pred, mass_target)

        calorie_loss = self.mae_loss(calorie_pred, calorie_target)

        total_loss = (self.ingredient_weight * ingredient_loss +
                      self.fat_weight * fat_loss +
                      self.carbs_weight * carbs_loss +
                      self.protein_weight * protein_loss +
                      self.mass_weight * mass_loss +
                      self.calorie_weight * calorie_loss)

        return total_loss, ingredient_loss, fat_loss, carbs_loss, protein_loss, mass_loss, calorie_loss

In [11]:
def calculate_ingredient_metrics(predictions, targets, threshold=0.5):
    pred_binary = (predictions > threshold).float()

    exact_match = (pred_binary == targets).all(dim=1).float().mean()

    element_accuracy = (pred_binary == targets).float().mean()

    true_positive = (pred_binary * targets).sum()
    false_positive = (pred_binary * (1 - targets)).sum()
    false_negative = ((1 - pred_binary) * targets).sum()

    precision = true_positive / (true_positive + false_positive + 1e-8)
    recall = true_positive / (true_positive + false_negative + 1e-8)
    f1_score = 2 * precision * recall / (precision + recall + 1e-8)

    return {
        'exact_match_accuracy': exact_match.item(),
        'element_accuracy': element_accuracy.item(),
        'precision': precision.item(),
        'recall': recall.item(),
        'f1_score': f1_score.item()
    }


def calculate_regression_metrics(predictions, targets):
    mae = torch.mean(torch.abs(predictions - targets))
    mse = torch.mean((predictions - targets) ** 2)
    rmse = torch.sqrt(mse)

    relative_error = torch.mean(
        torch.abs((predictions - targets) / (targets + 1e-8))) * 100

    return {
        'mae': mae.item(),
        'rmse': rmse.item(),
        'relative_error': relative_error.item()
    }

In [12]:
train_dish_ids = pd.read_csv(
    './dataset/dish_ids/splits/rgb_train_ids.txt',
    header=None,
    names=['dish_id'])

test_dish_ids = pd.read_csv(
    './dataset/dish_ids/splits/rgb_test_ids.txt',
    header=None,
    names=['dish_id'])


train_df = pd.merge(train_dish_ids, top5ingredientsByMass, on='dish_id')

test_df = pd.merge(test_dish_ids, top5ingredientsByMass, on='dish_id')

print(train_df.shape)
print(test_df.shape)

(12276, 13)
(2134, 13)


In [13]:
def train_multihead_model(train_loader, val_loader, model, criterion, optimizer, device, num_epochs=10):
    best_loss = float('inf')

    for epoch in range(num_epochs):
        model.train()

        running_losses = {
            'total': 0.0, 'ingredient': 0.0, 'fat': 0.0, 'carbs': 0.0,
            'protein': 0.0, 'mass': 0.0, 'calorie': 0.0
        }

        all_ingredient_preds, all_ingredient_targets = [], []
        all_fat_preds, all_fat_targets = [], []
        all_carbs_preds, all_carbs_targets = [], []
        all_protein_preds, all_protein_targets = [], []
        all_mass_preds, all_mass_targets = [], []
        all_calorie_preds, all_calorie_targets = [], []

        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("-" * 70)

        for batch_idx, (images, ingredient_labels, fat_labels, carbs_labels,
                        protein_labels, mass_labels, calorie_labels) in enumerate(train_loader):
            images = images.to(device)
            ingredient_labels = ingredient_labels.to(device)
            fat_labels = fat_labels.to(device)
            carbs_labels = carbs_labels.to(device)
            protein_labels = protein_labels.to(device)
            mass_labels = mass_labels.to(device)
            calorie_labels = calorie_labels.to(device)

            optimizer.zero_grad()

            ingredient_pred, fat_pred, carbs_pred, protein_pred, mass_pred, calorie_pred = model(
                images)

            total_loss, ingredient_loss, fat_loss, carbs_loss, protein_loss, mass_loss, calorie_loss = criterion(
                ingredient_pred, fat_pred, carbs_pred, protein_pred, mass_pred, calorie_pred,
                ingredient_labels, fat_labels, carbs_labels, protein_labels, mass_labels, calorie_labels
            )

            total_loss.backward()
            optimizer.step()

            running_losses['total'] += total_loss.item()
            running_losses['ingredient'] += ingredient_loss.item()
            running_losses['fat'] += fat_loss.item()
            running_losses['carbs'] += carbs_loss.item()
            running_losses['protein'] += protein_loss.item()
            running_losses['mass'] += mass_loss.item()
            running_losses['calorie'] += calorie_loss.item()

            all_ingredient_preds.append(ingredient_pred.detach())
            all_ingredient_targets.append(ingredient_labels)
            all_fat_preds.append(fat_pred.detach())
            all_fat_targets.append(fat_labels)
            all_carbs_preds.append(carbs_pred.detach())
            all_carbs_targets.append(carbs_labels)
            all_protein_preds.append(protein_pred.detach())
            all_protein_targets.append(protein_labels)
            all_mass_preds.append(mass_pred.detach())
            all_mass_targets.append(mass_labels)
            all_calorie_preds.append(calorie_pred.detach())
            all_calorie_targets.append(calorie_labels)

            if batch_idx % 10 == 0:
                print(f"Batch {batch_idx:4d}/{len(train_loader)} | "
                      f"Total: {total_loss.item():.4f} | "
                      f"Ingr: {ingredient_loss.item():.4f} | "
                      f"Fat: {fat_loss.item():.4f} | "
                      f"Carbs: {carbs_loss.item():.4f} | "
                      f"Protein: {protein_loss.item():.4f} | "
                      f"Mass: {mass_loss.item():.4f} | "
                      f"Cal: {calorie_loss.item():.4f}")

        avg_losses = {k: v / len(train_loader)
                      for k, v in running_losses.items()}

        all_ingredient_preds = torch.cat(all_ingredient_preds, dim=0)
        all_ingredient_targets = torch.cat(all_ingredient_targets, dim=0)
        all_fat_preds = torch.cat(all_fat_preds, dim=0)
        all_fat_targets = torch.cat(all_fat_targets, dim=0)
        all_carbs_preds = torch.cat(all_carbs_preds, dim=0)
        all_carbs_targets = torch.cat(all_carbs_targets, dim=0)
        all_protein_preds = torch.cat(all_protein_preds, dim=0)
        all_protein_targets = torch.cat(all_protein_targets, dim=0)
        all_mass_preds = torch.cat(all_mass_preds, dim=0)
        all_mass_targets = torch.cat(all_mass_targets, dim=0)
        all_calorie_preds = torch.cat(all_calorie_preds, dim=0)
        all_calorie_targets = torch.cat(all_calorie_targets, dim=0)

        ingredient_metrics = calculate_ingredient_metrics(
            all_ingredient_preds, all_ingredient_targets)
        fat_metrics = calculate_regression_metrics(
            all_fat_preds, all_fat_targets)
        carbs_metrics = calculate_regression_metrics(
            all_carbs_preds, all_carbs_targets)
        protein_metrics = calculate_regression_metrics(
            all_protein_preds, all_protein_targets)
        mass_metrics = calculate_regression_metrics(
            all_mass_preds, all_mass_targets)
        calorie_metrics = calculate_regression_metrics(
            all_calorie_preds, all_calorie_targets)

        print(f"\nEpoch {epoch+1} Training Summary:")
        print(f"Losses - Total: {avg_losses['total']:.4f} | Ingredient: {avg_losses['ingredient']:.4f} | "
              f"Fat: {avg_losses['fat']:.4f} | Carbs: {avg_losses['carbs']:.4f} | "
              f"Protein: {avg_losses['protein']:.4f} | Mass: {avg_losses['mass']:.4f} | Calorie: {avg_losses['calorie']:.4f}")

        print(f"Ingredient Metrics - Precision: {ingredient_metrics['precision']:.3f} | "
              f"Exact Match Acc: {ingredient_metrics['exact_match_accuracy']:.3f} | "
              f"Element Acc: {ingredient_metrics['element_accuracy']:.3f} | F1: {ingredient_metrics['f1_score']:.3f}")

        print(f"Fat Metrics - MAE: {fat_metrics['mae']:.4f} | "
              f"RMSE: {fat_metrics['rmse']:.4f} | Rel Error: {fat_metrics['relative_error']:.2f}%")

        print(f"Carbs Metrics - MAE: {carbs_metrics['mae']:.4f} | "
              f"RMSE: {carbs_metrics['rmse']:.4f} | Rel Error: {carbs_metrics['relative_error']:.2f}%")

        print(f"Protein Metrics - MAE: {protein_metrics['mae']:.4f} | "
              f"RMSE: {protein_metrics['rmse']:.4f} | Rel Error: {protein_metrics['relative_error']:.2f}%")

        print(f"Mass Metrics - MAE: {mass_metrics['mae']:.4f} | "
              f"RMSE: {mass_metrics['rmse']:.4f} | Rel Error: {mass_metrics['relative_error']:.2f}%")

        print(f"Calorie Metrics - MAE: {calorie_metrics['mae']:.4f} | "
              f"RMSE: {calorie_metrics['rmse']:.4f} | Rel Error: {calorie_metrics['relative_error']:.2f}%")

        if avg_losses['total'] < best_loss:
            best_loss = avg_losses['total']
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            torch.save({
                'model_state_dict': model.state_dict(),
                'mlb': train_loader.dataset.mlb,
                'scalers': train_loader.dataset.scalers,
                'epoch': epoch,
                'loss': best_loss,
                'approach': 'multihead_separate_macronutrients'
            }, f"multihead_separate_macros_model_{timestamp}.pth")
            print(f"✓ New best model saved! (Loss: {best_loss:.4f})")

    return model

In [14]:
def start_multihead_training():
    global top5ingredientsByMass, train_df, test_df
    img_dir = './dataset/side_angles'
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    train_transform, val_transform = create_transforms(416)

    full_dataset = DishDataset(top5ingredientsByMass, img_dir, train_transform)
    mlb = full_dataset.mlb
    scalers = full_dataset.scalers

    train_dataset = DishDataset(train_df, img_dir, train_transform, mlb=mlb)
    test_dataset = DishDataset(test_df, img_dir, val_transform, mlb=mlb)

    train_loader = DataLoader(
        train_dataset, batch_size=32, shuffle=True, num_workers=12, pin_memory=True)
    val_loader = DataLoader(test_dataset, batch_size=32,
                            shuffle=False, num_workers=12, pin_memory=True)

    print(
        f"Training with {len(train_dataset)} samples, {len(test_dataset)} test samples")
    print(f"Number of ingredients: {len(mlb.classes_)}")

    model = IngredientClassifier(
        len(mlb.classes_), model_variant='efficientnetv2_rw_m', freeze_backbone=False)
    # checkpoint = torch.load('multihead_model_20250601_150524.pth',
    #                    map_location=device,
    #                    weights_only=False)
    # model.load_state_dict(checkpoint['model_state_dict'])
    model.to(device)

    criterion = MultiHeadLoss(ingredient_weight=1.0,
                              fat_weight=1.0,
                              carbs_weight=1.0,
                              protein_weight=1.0,
                              mass_weight=1.0,
                              calorie_weight=1.0)

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

    trained_model = train_multihead_model(
        train_loader, val_loader, model, criterion, optimizer, device, num_epochs=15
    )

    return trained_model

In [ ]:
torch.cuda.empty_cache()
trained_model = start_multihead_training()

Using device: cuda
Dataset created with 75 ingredients
Using separate heads for fat, carbs, protein densities, total mass, and total calories
Dataset created with 75 ingredients
Using separate heads for fat, carbs, protein densities, total mass, and total calories
Dataset created with 75 ingredients
Using separate heads for fat, carbs, protein densities, total mass, and total calories
Training with 40523 samples, 6916 test samples
Number of ingredients: 75


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Detected feature dimension: 2152
Backbone parameters: 51,083,442
Frozen parameters: 0 (0.0%)
Trainable parameters: 51,083,442 (100.0%)
Model architecture (adapted from paper's multi-task design):
Backbone: efficientnetv2_rw_m
Input size: 416x416
Shared features: 2152 → 4096 → 4096
Ingredient head: 4096 → 2048 → 75
Fat head: 4096 → 4096 → 1024 → 1 (portion-independent density)
Carbs head: 4096 → 4096 → 1024 → 1 (portion-independent density)
Protein head: 4096 → 4096 → 1024 → 1 (portion-independent density)
Mass head: 4096 → 4096 → 1024 → 1
Calorie head: 4096 → 4096 → 1024 → 1

Epoch 1/15
----------------------------------------------------------------------
Batch    0/1267 | Total: 4.0339 | Ingr: 0.7043 | Fat: 0.7186 | Carbs: 0.3475 | Protein: 1.2183 | Mass: 0.4454 | Cal: 0.5997
Batch   10/1267 | Total: 3.7189 | Ingr: 0.6844 | Fat: 0.5012 | Carbs: 0.4523 | Protein: 0.9052 | Mass: 0.4639 | Cal: 0.7119
Batch   20/1267 | Total: 3.3343 | Ingr: 0.6632 | Fat: 0.4984 | Carbs: 0.3858 | Protein:

# Evaluation